In [4]:
from ultralytics import YOLO
import cv2
import time

# Paths to the YOLOv8 model weights
models = {
    'YOLOv8n': 'yolov8n.pt',  # nano - fastest, smallest
    'YOLOv8s': 'yolov8s.pt',  # small
    'YOLOv8m': 'yolov8m.pt'   # medium - more accurate, slower
}

video_path = '/home/sayeed/no_face_video.mp4'

# Iterate over models to evaluate each
for name, weight in models.items():
    print(f"\n🔍 Running {name}...")

    # Load model
    model = YOLO(weight)

    # Open video
    cap = cv2.VideoCapture(video_path)
    width, height = int(cap.get(3)), int(cap.get(4))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(f'{name}_output.mp4', fourcc, 30, (width, height))

    frame_count = 0
    total_time = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        start = time.time()
        results = model(frame)  # Run YOLOv8 detection
        end = time.time()

        # Timing
        inference_time = end - start
        total_time += inference_time
        frame_count += 1

        # Annotated frame
        annotated_frame = results[0].plot()
        out.write(annotated_frame)

        # Optional: show in real time
        cv2.imshow(name, annotated_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    avg_fps = frame_count / total_time
    print(f"✅ {name} done. Processed {frame_count} frames.")
    print(f"⏱️ Average FPS: {avg_fps:.2f}")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/sayeed/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

🔍 Running YOLOv8n...


100%|██████████| 6.25M/6.25M [00:05<00:00, 1.12MB/s]



0: 384x640 (no detections), 105.7ms
Speed: 5.0ms preprocess, 105.7ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)



qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/sayeed/Documents/Deep_learning/env/lib/python3.12/site-packages/cv2/qt/plugins"


0: 384x640 (no detections), 138.5ms
Speed: 10.7ms preprocess, 138.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 68.2ms
Speed: 10.8ms preprocess, 68.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 58.6ms
Speed: 3.2ms preprocess, 58.6ms inference, 4.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 127.3ms
Speed: 4.2ms preprocess, 127.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 49.0ms
Speed: 5.8ms preprocess, 49.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 orange, 65.3ms
Speed: 2.3ms preprocess, 65.3ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 50.1ms
Speed: 1.9ms preprocess, 50.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 67.7ms
Speed: 3.0ms preprocess, 67.7ms inference,

100%|██████████| 21.5M/21.5M [00:07<00:00, 3.11MB/s]



0: 384x640 (no detections), 119.7ms
Speed: 1.5ms preprocess, 119.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 141.3ms
Speed: 12.3ms preprocess, 141.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 123.5ms
Speed: 2.3ms preprocess, 123.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 120.3ms
Speed: 1.5ms preprocess, 120.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 125.5ms
Speed: 1.7ms preprocess, 125.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 113.5ms
Speed: 1.6ms preprocess, 113.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 122.1ms
Speed: 1.6ms preprocess, 122.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 119.9ms
Speed: 8.5ms prep

100%|██████████| 49.7M/49.7M [00:52<00:00, 991kB/s] 



0: 384x640 (no detections), 297.8ms
Speed: 1.8ms preprocess, 297.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 350.9ms
Speed: 12.6ms preprocess, 350.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 336.2ms
Speed: 11.5ms preprocess, 336.2ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 257.3ms
Speed: 3.2ms preprocess, 257.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 253.9ms
Speed: 6.9ms preprocess, 253.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 276.9ms
Speed: 2.3ms preprocess, 276.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 418.5ms
Speed: 1.4ms preprocess, 418.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 302.3ms
Speed: 1.6ms preprocess, 302.3